In [1]:
# Обработка данных
import numpy as np
from numpy.polynomial import Polynomial
import pandas as pd
from os import listdir
# Визуализация
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

## Метод QR-разложения

https://en.wikipedia.org/wiki/QR_decomposition

https://stats.stackexchange.com/questions/20643/finding-matrix-eigenvectors-using-qr-decomposition

In [2]:
def QR(A):
    
    def proj(u, a):
        return np.inner(u, a) / np.inner(u, u) * u
    
    k = A.shape[0]
    Q = np.zeros(A.shape)
    a = A.T
    for i in range(k):
        Q[i, :] = a[i] - sum(
            proj(Q[j], a[i]) 
            for j in range(i)
        )

    Q = Q.T / np.linalg.norm(Q, axis=1)
    R = np.dot(Q.T, A)
    return Q, R


def QR_eigen(A, n_iter=100):
    X = A.copy()
    qq = np.eye(A.shape[0])

    for i in range(n_iter):
        Q, R = np.linalg.qr(X)
        qq = np.dot(qq, Q)
        X = np.dot(R, Q)

    return np.diagonal(X), qq

## Выбор данных

Источник: https://data.worldbank.org/indicator

Мотивация: сжатие данных позволит выделить основные характеристики стран

Гипотеза: первая главная компонента будет соответствовать развитости страны

In [3]:
DATA_DIR = 'data/world_bank_data/'

In [4]:
def read_data(path):
    df = pd.DataFrame()
    for file in sorted(listdir(path)):
        # Читаем файл
        series = pd.read_excel(
            DATA_DIR+file, 
            skiprows=3,
            # Индексом используем код страны,
            # потому что он используется для карты
            index_col='Country Code'
        )
        # Берем среднее за 2017-2019 год, преимущества:
        # 1) Меньше пропущенных значений
        # 2) Небольшое сглаживание 
        series = series.loc[:, '2017':'2019'].mean(axis=1)
        df[file] = series
    df.columns = df.columns.str.replace('.xls', '', regex=False)
    return df

In [5]:
sorted(listdir(DATA_DIR))

['Adolescent fertility.xls',
 'Fertility rate.xls',
 'GDP per capita PPP.xls',
 'Life expectancy.xls',
 'Population ages 0-14.xls',
 'Population ages 65+.xls',
 'Population growth.xls',
 'Urban population.xls',
 'Working in agriculture.xls',
 'Working in industry.xls']

Использованные признаки:

* ВВП на душу по покупательной способности
* Ожидаемая продолжительность жизни
* Процент городского населения
* Общая рождаемость
* Рост населения, %
* Рождаемость среди подростков (15-19)
* Процент занятых в сельском хозяйстве
* Процент занятых в промышленности
* Население младше 14, %
* Население старше 65, %



In [6]:
df = read_data(DATA_DIR)

In [7]:
# Так как пропуски неслучайные, их просто удалим
df = df.dropna()

In [8]:
df.shape

(223, 10)

In [9]:
# Часть зависимостей получалась нелинейной, для них применим логарифм
df['GDP per capita PPP'] = df['GDP per capita PPP'].apply(np.log)
df['Population ages 65+'] = df['Population ages 65+'].apply(np.log)

In [10]:
# Нормализация
# Вычесть среднее нужно для разложения
# Деление на std убирает разницу единиц измерения
df_norm = (df - df.mean()) / df.std()

## Применение PCA

In [11]:
def cov(X):
    return np.dot(X, X.T) / (X.shape[1] - 1)

In [12]:
X = df_norm.values
cov_matrix = cov(X.T)

In [13]:
%%time
QR_eigen(cov_matrix)

CPU times: total: 15.6 ms
Wall time: 10 ms


(array([7.33407736, 0.91814861, 0.7067822 , 0.37059063, 0.17744381,
        0.15631407, 0.14018248, 0.10485524, 0.06639169, 0.02521391]),
 array([[ 0.3223444 ,  0.08318889,  0.19005476, -0.57314822, -0.48209276,
         -0.43030734, -0.29211821,  0.14092405,  0.0349189 , -0.02375285],
        [ 0.34872139,  0.16418408, -0.01863818, -0.10736653, -0.22222192,
          0.49074181,  0.27016395, -0.19748893,  0.4153561 , -0.51462684],
        [-0.34187657,  0.25050102,  0.09562972,  0.05654197, -0.05397934,
         -0.02062082, -0.44750156, -0.53111159,  0.52087223,  0.22592827],
        [-0.33888475,  0.07627114,  0.1043441 ,  0.40247167, -0.43585069,
         -0.15939703,  0.19618887,  0.55099933,  0.38175296, -0.05718225],
        [ 0.35757053,  0.1173973 , -0.05209595, -0.0657474 , -0.020014  ,
          0.35596006,  0.11103611,  0.24312491,  0.21209679,  0.77961053],
        [-0.31697031, -0.32085071,  0.3174507 , -0.13111959, -0.52941592,
          0.27396738,  0.28505959, -0.31818

In [14]:
from sklearn.decomposition import PCA
check = PCA().fit(X)
check.explained_variance_, check.components_.T

(array([7.33407736, 0.91814861, 0.7067822 , 0.37059063, 0.17744381,
        0.15631407, 0.14018248, 0.10485524, 0.06639169, 0.02521391]),
 array([[ 0.3223444 ,  0.08318889, -0.19005476, -0.57314822,  0.48209455,
          0.43030323,  0.2921213 , -0.14092405, -0.0349189 ,  0.02375285],
        [ 0.34872139,  0.16418408,  0.01863818, -0.10736653,  0.22221987,
         -0.4907408 , -0.27016747,  0.19748893, -0.4153561 ,  0.51462684],
        [-0.34187657,  0.25050102, -0.09562972,  0.05654197,  0.05397943,
          0.02061739,  0.4475017 ,  0.53111159, -0.52087223, -0.22592827],
        [-0.33888475,  0.07627114, -0.1043441 ,  0.40247167,  0.43585135,
          0.15939661, -0.19618773, -0.55099933, -0.38175296,  0.05718225],
        [ 0.35757053,  0.1173973 ,  0.05209595, -0.0657474 ,  0.02001251,
         -0.35595934, -0.11103866, -0.24312491, -0.21209679, -0.77961053],
        [-0.31697031, -0.32085071, -0.3174507 , -0.13111959,  0.52941477,
         -0.27396755, -0.28506155,  0.31818

## Остальное - аналогично ноутбуку 3.1